<a href="https://colab.research.google.com/github/djaiello/NYPD_Arrests_Final_Project/blob/machine_learning_model/Supervised_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn. datasets import load_digits
digits = load_digits()
from google.colab import files
from google.colab import drive

# Initial imports
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Mount the Google Drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# Full dataset (~4 min.)
arrest_df = pd.read_csv("gdrive/My Drive/Colab Notebooks/arrest_10yr_clean.csv")

print(arrest_df.shape)
arrest_df.head()

(2855141, 16)


,ARREST_KEY,ARREST_DATE,PD_CD,KY_CD,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,82422385,2012-01-01,759.0,359.0,PL 1950500,M,K,79,18-24,M,BLACK,997366.0,192877.0,40.696078,-73.952702,POINT (-73.95270199199997 40.69607807800003)
1,82422694,2012-01-01,101.0,344.0,PL 1200001,M,M,6,18-24,M,WHITE,983794.0,204737.0,40.728641,-74.001647,POINT (-74.00164669699996 40.72864066600005)
2,82422331,2012-01-01,905.0,347.0,VTL11920U2,M,K,72,25-44,M,WHITE HISPANIC,989519.0,178298.0,40.656070,-73.981012,POINT (-73.981011557 40.656070154000076)
3,82422817,2012-01-01,782.0,236.0,PL 2650101,M,Q,115,25-44,M,BLACK HISPANIC,1017897.0,219850.0,40.770058,-73.878527,POINT (-73.87852744499997 40.77005803100008)
4,82422890,2012-01-01,258.0,351.0,PL 1456002,M,Q,104,18-24,M,WHITE,1012109.0,195368.0,40.702881,-73.899525,POINT (-73.89952491499997 40.70288116000006)


In [3]:
arrest_df['ARREST_BORO'].value_counts()

K    794319
M    760908
B    634965
Q    558602
S    106347
Name: ARREST_BORO, dtype: int64

In [4]:
columns_drop = ["X_COORD_CD", "Y_COORD_CD", "Latitude", "Longitude", "Lon_Lat", "ARREST_KEY", "ARREST_DATE", "LAW_CODE"]
arrest_df = arrest_df.drop(columns_drop, 1)
arrest_df.head()

<ipython-input-4-7bf7e7903fc0>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  arrest_df = arrest_df.drop(columns_drop, 1)


,PD_CD,KY_CD,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,AGE_GROUP,PERP_SEX,PERP_RACE
0,759.0,359.0,M,K,79,18-24,M,BLACK
1,101.0,344.0,M,M,6,18-24,M,WHITE
2,905.0,347.0,M,K,72,25-44,M,WHITE HISPANIC
3,782.0,236.0,M,Q,115,25-44,M,BLACK HISPANIC
4,258.0,351.0,M,Q,104,18-24,M,WHITE


In [5]:
arrest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2855141 entries, 0 to 2855140
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   PD_CD            float64
 1   KY_CD            float64
 2   LAW_CAT_CD       object 
 3   ARREST_BORO      object 
 4   ARREST_PRECINCT  int64  
 5   AGE_GROUP        object 
 6   PERP_SEX         object 
 7   PERP_RACE        object 
dtypes: float64(2), int64(1), object(5)
memory usage: 174.3+ MB


In [6]:
arrest_df["AGE_GROUP"].value_counts()

25-44    1382728
18-24     707682
45-64     554451
<18       180976
65+        29304
Name: AGE_GROUP, dtype: int64

In [7]:
arrest_df["AGE_GROUP"]= arrest_df["AGE_GROUP"].str.replace("25-", "")
arrest_df["AGE_GROUP"]= arrest_df["AGE_GROUP"].str.replace("18-", "")
arrest_df["AGE_GROUP"]= arrest_df["AGE_GROUP"].str.replace("45-", "")
arrest_df["AGE_GROUP"]= arrest_df["AGE_GROUP"].str.replace("<18", "17")
arrest_df["AGE_GROUP"]= arrest_df["AGE_GROUP"].str.replace("+", "")
arrest_df["AGE_GROUP"].value_counts()

<ipython-input-7-82071f55752f>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  arrest_df["AGE_GROUP"]= arrest_df["AGE_GROUP"].str.replace("+", "")


44    1382728
24     707682
64     554451
17     180976
65      29304
Name: AGE_GROUP, dtype: int64

In [8]:
# Encode letters to number

arrest_df_encoded = pd.get_dummies(arrest_df, columns=["PERP_SEX", "PERP_RACE", "LAW_CAT_CD"])
arrest_df_encoded.head()

,PD_CD,KY_CD,ARREST_BORO,ARREST_PRECINCT,AGE_GROUP,PERP_SEX_F,PERP_SEX_M,PERP_RACE_AMERICAN INDIAN/ALASKAN NATIVE,PERP_RACE_ASIAN / PACIFIC ISLANDER,PERP_RACE_BLACK,PERP_RACE_BLACK HISPANIC,PERP_RACE_UNKNOWN,PERP_RACE_WHITE,PERP_RACE_WHITE HISPANIC,LAW_CAT_CD_F,LAW_CAT_CD_I,LAW_CAT_CD_M,LAW_CAT_CD_V
0,759.0,359.0,K,79,24,0,1,0,0,1,0,0,0,0,0,0,1,0
1,101.0,344.0,M,6,24,0,1,0,0,0,0,0,1,0,0,0,1,0
2,905.0,347.0,K,72,44,0,1,0,0,0,0,0,0,1,0,0,1,0
3,782.0,236.0,Q,115,44,0,1,0,0,0,1,0,0,0,0,0,1,0
4,258.0,351.0,Q,104,24,0,1,0,0,0,0,0,1,0,0,0,1,0


In [9]:
# Define the features set.
X = arrest_df_encoded.copy()
X = X.drop("ARREST_BORO", axis=1)

# Define the target set.
y = arrest_df_encoded["ARREST_BORO"].values

X_train, X_test, y_train, y_test = train_test_split( X,y, random_state=42)

In [10]:
print(X_train.shape)
print(y_train.shape)

(2141355, 17)
(2141355,)


In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
model = RandomForestClassifier(n_estimators=200)
model.fit(X_train, y_train )

RandomForestClassifier(n_estimators=200)

In [13]:
from sklearn.metrics import confusion_matrix

In [14]:
y_preds = model.predict(X_test)

In [15]:
print(y_preds)

['K' 'B' 'M' ... 'K' 'Q' 'M']


In [16]:
# Confusion Matrix
confusion_matrix(y_test, y_preds)

array([[158980,      0,      1,      0,      0],
       [     1, 198472,      0,      0,      0],
       [     1,      0, 190481,     97,      0],
       [     0,      1,      5, 139368,      0],
       [     0,      0,      0,      0,  26379]])